# Regression Model

In [98]:
import pandas as pd
import numpy as np
import missingno as msno

import warnings
warnings.filterwarnings('ignore')

## import data

In [3]:
df_click_purchase = pd.read_csv('data/clickstreams_with_purchase.csv')
df_click_purchase.head()

,customer_id,date,time,page_viewed,product_id,ordered
0,1436364061643577713,2018-07-16,14:37:44,HOMEPAGE,0,0.0
1,1436364061643577713,2018-07-16,14:40:44,PRODUCTSIMPLESKU,205401096165894320,0.0
2,1436364061643577713,2018-07-16,14:42:39,PRODUCTSIMPLESKU,9220344733964903803,0.0
3,1436364061643577713,2018-07-16,14:42:53,PRODUCTSIMPLESKU,9220344733964903803,0.0
4,1436364061643577713,2018-07-16,14:44:46,KEYWORDSEARCH,0,0.0


In [92]:
df_orders = pd.read_csv('data/orders.csv')
print(df_orders.shape)
df_orders.head()

(137537, 9)


,order_product_id,order_id,wayfair_product_id,wayfair_website_id,order_date,order_time,customer_id,customer_zip,quantity_ordered
0,8.344710e+18,1.073040e+17,8.089580e+18,Wayfair,7/19/18,9:48:02,3.243950e+18,29464,1
1,7.897050e+18,9.115090e+17,4.407090e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
2,3.742700e+17,9.115090e+17,1.327620e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
3,7.072690e+18,5.760940e+18,7.293640e+18,Wayfair,7/22/18,21:56:42,3.106940e+18,6712,1
4,7.307240e+18,2.708890e+18,7.264500e+18,Wayfair,7/21/18,12:55:20,1.390570e+18,2468,1


In [94]:
df_orders1 = df_orders.rename(columns={'wayfair_product_id':'product_id'})
df_orders1.head()

,order_product_id,order_id,product_id,wayfair_website_id,order_date,order_time,customer_id,customer_zip,quantity_ordered
0,8.344710e+18,1.073040e+17,8.089580e+18,Wayfair,7/19/18,9:48:02,3.243950e+18,29464,1
1,7.897050e+18,9.115090e+17,4.407090e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
2,3.742700e+17,9.115090e+17,1.327620e+18,Wayfair,7/21/18,15:04:23,7.606760e+18,10928,1
3,7.072690e+18,5.760940e+18,7.293640e+18,Wayfair,7/22/18,21:56:42,3.106940e+18,6712,1
4,7.307240e+18,2.708890e+18,7.264500e+18,Wayfair,7/21/18,12:55:20,1.390570e+18,2468,1


In [60]:
df_products = pd.read_csv('data/products.csv')
print(df_products.shape)
df_products.head()

(498673, 13)


,website_name,product_id,product_name,category_name,description,manufacturer_name,weight,dimension_1,dimension_2,dimension_3,num_reviews,onsite_price,on_promotion
0,Joss US,2.790000e+18,Westgate Upholstered Panel Bed,Beds,The 1882 upholstered bed is sure to appeal to ...,Pulaski Furniture,116.0,59.25,87.50,68.75,7.0,409.99,0
1,Joss US,8.190000e+18,3 Piece Leah Bowl Set,"Boxes, Bins, Baskets, & Buckets",Stack green apples on the breakfast bar or dis...,Three Hands Co.,30.8,23.00,28.35,14.17,0.0,89.99,1
2,Joss US,7.240000e+18,Butter Cups/Ranunculus Sunshine Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,16.00,3.00,4.00,0.0,16.99,0
3,Joss US,4.110000e+18,Freesias Double Blooming Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,24.00,2.00,2.00,2.0,9.99,0
4,Joss US,7.670000e+18,Grand Freesia/Ranunculus Blend Bulb,Planting & Seeding,Start your own backyard garden or add spring s...,Van Zyverden Inc,1.0,36.00,3.00,2.00,2.0,9.99,0


In [90]:
columns = ['product_id','weight','num_reviews','onsite_price','on_promotion']
df_products1 = df_products.loc[:,columns]
df_products1.head()

,product_id,weight,num_reviews,onsite_price,on_promotion
0,2.790000e+18,116.0,7.0,409.99,0
1,8.190000e+18,30.8,0.0,89.99,1
2,7.240000e+18,1.0,0.0,16.99,0
3,4.110000e+18,1.0,2.0,9.99,0
4,7.670000e+18,1.0,2.0,9.99,0


In [105]:
df = pd.merge(df_orders1, df_products1, on='product_id')
df = df.drop(columns=['order_product_id','order_id','wayfair_website_id'])
df.head()

,product_id,order_date,order_time,customer_id,customer_zip,quantity_ordered,weight,num_reviews,onsite_price,on_promotion
0,7.842510e+15,7/19/18,13:00:39,6.231540e+18,61832,1,62.0,511.0,157.31,1
1,7.842510e+15,7/18/18,17:10:51,3.839240e+18,8858,1,62.0,511.0,157.31,1
2,7.842510e+15,7/21/18,10:48:45,4.570650e+18,28054,1,62.0,511.0,157.31,1
3,7.842510e+15,7/18/18,18:09:05,2.495660e+18,78233,1,62.0,511.0,157.31,1
4,7.842510e+15,7/22/18,10:05:55,7.211280e+18,7036,1,62.0,511.0,157.31,1


In [104]:
df.shape

(80123, 10)

In [107]:
df['order_value'] = df.quantity_ordered*df.onsite_price
df.head()

,product_id,order_date,order_time,customer_id,customer_zip,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,order_value
0,7.842510e+15,7/19/18,13:00:39,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31
1,7.842510e+15,7/18/18,17:10:51,3.839240e+18,8858,1,62.0,511.0,157.31,1,157.31
2,7.842510e+15,7/21/18,10:48:45,4.570650e+18,28054,1,62.0,511.0,157.31,1,157.31
3,7.842510e+15,7/18/18,18:09:05,2.495660e+18,78233,1,62.0,511.0,157.31,1,157.31
4,7.842510e+15,7/22/18,10:05:55,7.211280e+18,7036,1,62.0,511.0,157.31,1,157.31


In [120]:
df.customer_zip=df.customer_zip.astype(int)
df.customer_zip.dtype

dtype('int32')

In [123]:
df_taxes = pd.read_csv('data/taxes.csv')
df_taxes.head()

,state_fips,state,zipcode,agi_stub,num_returns,num_single,num_joint,num_hoh,num_dependents,num_elderly,...,N19300,A19300,N19500,A19500,N04800,A04800,N07220,A07220,N11901,A11901
0,1,AL,0,1,815440,477700,105350,221200,491310,150920,...,25020,5950,46930,74541,17910,82471,27490,15002,0,0
1,1,AL,0,2,495830,211930,142340,128890,360480,109680,...,64360,17611,102740,243301,44430,292180,43580,45248,0,0
2,1,AL,0,3,263390,83420,137870,36340,182880,75570,...,68320,19479,99920,337719,37110,278272,22780,29166,0,0
3,1,AL,0,4,167190,29420,124060,10610,130160,51540,...,58540,18167,79980,362811,24780,202113,17250,22630,0,0
4,1,AL,0,5,217440,20240,188080,4880,195990,68080,...,117980,44270,151920,1065202,34030,298272,20970,29469,390,155


In [125]:
df_taxes1 = df_taxes.loc[:,['state','zipcode','agi_stub']]
df_taxes1.head()

,state,zipcode,agi_stub
0,AL,0,1
1,AL,0,2
2,AL,0,3
3,AL,0,4
4,AL,0,5


In [127]:
df1 = pd.merge(df,df_taxes1,left_on='customer_zip',right_on='zipcode')
df1.head()

,product_id,order_date,order_time,customer_id,customer_zip,quantity_ordered,weight,num_reviews,onsite_price,on_promotion,order_value,state,zipcode,agi_stub
0,7.842510e+15,7/19/18,13:00:39,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31,IL,61832,1
1,7.842510e+15,7/19/18,13:00:39,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31,IL,61832,2
2,7.842510e+15,7/19/18,13:00:39,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31,IL,61832,3
3,7.842510e+15,7/19/18,13:00:39,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31,IL,61832,4
4,7.842510e+15,7/19/18,13:00:39,6.231540e+18,61832,1,62.0,511.0,157.31,1,157.31,IL,61832,5
